In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import glm
import statsmodels.api as sm
import re
import random
import time
from imblearn.over_sampling import SMOTE
import scipy.stats
import scipy.optimize as so
from sklearn.linear_model import LogisticRegression
from prettytable import PrettyTable
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import warnings
warnings.filterwarnings("ignore")

# Data Loading and Cleaning 

In [2]:
#Upload Data and assign Index as first column and add Binary 1/0 for Final Column as 'label'

df_recipe = pd.read_csv('final_data_set.csv',index_col=0)
df_recipe['label']=df_recipe['protein']/df_recipe['calories']
df_recipe['label'] = np.where(df_recipe['label']>0.1, 1, 0)
df_recipe.head(5)

,title,calories,protein,advance prep required,alabama,alaska,alcoholic,almond,amaretto,anchovy,...,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey,label
0,Rhubarb Roulade,256.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,Caramel Macadamia Nut Crunch,223.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Grand Marnier Brownie Kisses,195.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,Herb Salad Spring Rolls with Spicy Peanut Sauce,224.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,Festival,219.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
np.random.seed(1)

In [4]:
#The number of each class label

df_recipe.label.value_counts()

0    10061
1      532
Name: label, dtype: int64

In [5]:
#Index of columns and length 

df_recipe.columns

Index(['title', 'calories', 'protein', 'advance prep required', 'alabama',
       'alaska', 'alcoholic', 'almond', 'amaretto', 'anchovy',
       ...
       'yogurt', 'yonkers', 'yuca', 'zucchini', 'cookbooks', 'leftovers',
       'snack', 'snack week', 'turkey', 'label'],
      dtype='object', length=673)

In [6]:
#How many columns include any null values?

null_columns = df_recipe.columns[df_recipe.isnull().any()]
df_recipe[null_columns].isnull().sum()

Series([], dtype: float64)

In [7]:
#What are the different data types?

df_recipe.dtypes

title                     object
calories                 float64
protein                  float64
advance prep required    float64
alabama                  float64
                          ...   
leftovers                float64
snack                    float64
snack week               float64
turkey                   float64
label                      int32
Length: 673, dtype: object

In [8]:
#Replace any special characters with underscore (_)

df_recipe.columns = df_recipe.columns.str.replace('[^a-zA-Z0-9]', '_', regex=True)

In [9]:
#Now we will restrict of data to all recipes and the Label. Remove Title, Calories, and Protein

data = df_recipe.iloc[:,3:].copy()
data.head(5)

,advance_prep_required,alabama,alaska,alcoholic,almond,amaretto,anchovy,anise,anniversary,anthony_bourdain,...,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack_week,turkey,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
# Balance the dataset using the SMOTE() function to even the 

oversample = SMOTE(random_state=0)
X, y = oversample.fit_resample(data.iloc[:,:-1], data.iloc[:,-1])
data_balance = pd.concat([X, y],axis=1)
data_balance.tail(5)

,advance_prep_required,alabama,alaska,alcoholic,almond,amaretto,anchovy,anise,anniversary,anthony_bourdain,...,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack_week,turkey,label
20117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179044,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.820956,1
20118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
20119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1
20120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
20121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1


In [11]:
data_balance.label.value_counts()

0    10061
1    10061
Name: label, dtype: int64

# Functions and Model Selection

### Functions 

In [12]:
"""Here we define a predict function that will try to predict whether an object is class 1 or 0 based on
our sigmoid transform. We will use an input Beta that will act as coefficients to our vector elements and 
then using different threshold to tune the best threshold we will predict the class"""

def predict(df, Beta, threshold):
    pred_linear = np.dot(np.array(df), Beta)
    # using sigmoid tranform to logistic
    predictions = 1 / (1 + np.exp(-pred_linear))
    predictions = np.where(predictions > threshold, 1, 0)
    return predictions

def std_revenue(revenues_list):
    # Standard deviation of revenues list
    # Using sum() + list comprehension
    mean = sum(revenues_list) / len(revenues_list)
    variance = sum([((x - mean) ** 2) for x in revenues_list]) / (len(revenues_list)-1)
    res = variance ** 0.5
    std_error_revenue = np.round(res/(len(revenues_list)),2)
    return std_error_revenue

"""Using hp_predictions_and_actuals as an imput parameter the following function is given so that 
we can calculate Revenue"""

def simulate(hp_predictions_and_actuals):
    high_protein_ad_revenue = 1
    low_protein_ad_revenue = .25
    
    ad_revenue = 0
    analysis_queue = []
    active_recipes = []
    i_data = 0
    for _ in range(365):  # for one year
        # 50-100 recipe submissions/day
        num_submissions = np.random.randint(50, 100)
        for i in range(num_submissions): # 
            if i_data < len(hp_predictions_and_actuals):
                p, a = hp_predictions_and_actuals[i_data]
                if p == 1:  # go to the front of the queue
                    analysis_queue.insert(0, a)
                    
                else: # go to the back of the queue
                    analysis_queue.append(a)
                i_data += 1
            
        # can analyze only 25 recipes/day   
        for __ in range(25):
            if len(analysis_queue)==0:
                break
            acutal = analysis_queue.pop(0)
            active_recipes.append(acutal)     
        # run 500-1000 ads/day
        num_ads_today = np.random.randint(500, 1000)
        for a in random.choices(active_recipes, k=num_ads_today):
            # a==1 if recipe is high protein
            if a==1:
                ad_revenue += high_protein_ad_revenue
            else:
                ad_revenue += low_protein_ad_revenue
    
    return ad_revenue

# Model Comparison Metric Lists

In [13]:
#Create Lists for Final Comparison
revenues = []
thresholds = []
accuracys = []
times = []
n_regressors = []
std_err_revenues = []

### Method 1: Random Selection

In [14]:
#Randomly select 100 regressors to build model
random_columns = random.sample(list(data_balance.columns), 100)
Train_data = pd.concat([data_balance[random_columns],data_balance.label], axis = 1)

#Build a model formula with all regressors, excluding the intercept
start1 = time.time()
all_columns = "+".join(Train_data.columns.drop('label'))
my_formula = "label~" + all_columns + '-1' 

#Using GLM and Binomial fit, we identify significant columns by p-values

res_poly = glm(formula=my_formula, data=Train_data, family=sm.families.Binomial()).fit()

signi_detail = res_poly.pvalues[res_poly.pvalues < 0.05]
signi_columns = signi_detail.index

random_columns = res_poly.params.index
random_B = res_poly.params.values

""" Select the GLM columns and their values to apply of predict() function to create an array, then we
zip them with the actual class from the y (the label column). Here we will get an array of pairs to feed 
into the simulate() function. From there, we can calculate the Revenue of our Model #1 """

x = data.iloc[:, :-1]
y = data.iloc[:,-1]

# predict and evaluate the result
threshold_candidate = np.arange(0.1,1,0.1)
pre_revenue = -np.inf
for threshold in threshold_candidate:
    predictions = predict(x[random_columns], random_B, threshold)
    actuals = np.array(y).copy()
    hp_predictions_and_actuals = list(zip(predictions, actuals))
    revenue = simulate(hp_predictions_and_actuals)
    if revenue > pre_revenue:
        pre_revenue = revenue
        model1_best_threshold = np.round(threshold,1)
        model1_predictions = predictions
        final_hp_predictions_and_actuals = hp_predictions_and_actuals

model1_revenue = np.round(pre_revenue,2)
model1_accuracy = np.round(len(model1_predictions[model1_predictions==actuals])/len(model1_predictions),2)
end1 = time.time()

revenues_list = [simulate(final_hp_predictions_and_actuals) for _ in range(1000)]
model1_revenue_std_error = std_revenue(revenues_list)

model1_time = np.round(end1-start1,2)

n_regressors.append(len(random_columns))
revenues.append(model1_revenue)
thresholds.append(model1_best_threshold)
accuracys.append(model1_accuracy)
times.append(model1_time)
std_err_revenues.append(model1_revenue_std_error)

print('\nModel 1 parameters standard error: \n{}'.format(res_poly.bse))
print('\nModel 1 parameters standard error < 1: \n{}'.format(res_poly.bse[res_poly.bse<1]))
print('\nThe Number of Selected Regressors = {}'.format(len(random_columns)))
print('\nModel 1 Accuracy: {} '.format(model1_accuracy))
print('\nModel 1 Best threshold: {} '.format(model1_best_threshold))
print('\nModel 1 Time: {}s'.format(model1_time))
print('\nModel 1 Revenue: ${} '.format(model1_revenue))
print('\nModel 1 Revenue Standard Error: {} '.format(model1_revenue_std_error))


Model 1 parameters standard error: 
chestnut               3.755319e+05
sourdough              2.655035e+06
rosemary               1.315083e-01
game                   3.806022e-01
semolina               7.875883e+05
                           ...     
macaroni_and_cheese    2.636263e+06
kahl_a                 6.920765e+05
graduation             3.691792e-01
hamburger              1.805612e+06
orange                 8.987992e-02
Length: 100, dtype: float64

Model 1 parameters standard error < 1: 
rosemary           1.315083e-01
game               3.806022e-01
duck               2.191133e-01
noodle             2.195175e-01
dairy              9.253052e-02
easter             2.063452e-01
low_cholesterol    2.146966e-01
veal               1.854784e-01
low_fat            1.370340e-01
plum               3.651362e-01
lemongrass         2.440959e-01
eggplant           2.837167e-01
rum                1.849943e-01
hazelnut           3.622125e-01
france             5.679224e-01
grill             

### Method 2: Significant Regressors

In [15]:
""" Select the last 100 columns as Regressors and thee we refit with only the significant regressors as
our model training """

x = data_balance.iloc[:, -101:-1]
y = data_balance.iloc[:,-1]

""" Define the log likelihood function and logistic regression fit """

def log_likelihood(X, y, B):
    X = np.asarray(X)
    y = np.asarray(y).flatten()
    B = np.asarray(B).flatten()
    pi = np.maximum(1e-9, np.minimum(1-1e-9, 1 / (1 + np.exp(-X@B))))
    ll = (y*np.log(pi) + (1-y)*np.log(1-pi)).sum()
    return ll

def logistic_regression_fit(y, X):
    num_regressors = X.shape[1]
    res = so.minimize(lambda B: -log_likelihood(X, y, B), [0]*num_regressors)    
    B = res.x
    return pd.DataFrame(data=B[:,None].T, columns=X.columns).T

#Calculate the Coefficients of our Beta Vector with logistic regresison fit function
start2 = time.time()
coefficients = logistic_regression_fit(y, x)
B = coefficients.values.reshape(1,-1)[0]

#Calculate t-score to verify if the coefficients are significantly different from zero
t_scores = B/(np.std(x)/np.sqrt(len(data)))

#Create List of the p-values from statsmodel 
p_values = scipy.stats.t.sf(np.abs(t_scores), df=len(data))*2

#Use a significance level of α=.05 with and Confidence=.95 and append into list
alpha = 0.05
index = 0
predictor_index = []
for p in p_values:
    if p<alpha:
        predictor_index.append(index)
    index+=1

""" Apply the Logistic Regression Fit function to this new dataframe and our original label vector and 
identify the significant regressors to apply to simulate function """

signi_X = data.iloc[:,predictor_index]
y = data.iloc[:,-1]
coefficients_signi = logistic_regression_fit(y, signi_X)
B_signi = coefficients_signi.values.reshape(1,-1)[0]
Model2_SE_B = ((signi_X**2*np.exp(B_signi*signi_X)/((1+np.exp(B_signi*signi_X))**2)).sum())**(-1/2)

""" Reapply our GLM function to new reduced data, acquire an array and zip to our original y vector
then apply simulate function to obtain our new revenue estimate"""

# predict and evaluate the result
threshold_candidate = np.arange(0.1,1,0.1)
pre_revenue = -np.inf
for threshold in threshold_candidate:
    predictions = predict(signi_X, B_signi, threshold)
    actuals = np.array(y).copy()
    hp_predictions_and_actuals = list(zip(predictions, actuals))
    revenue = simulate(hp_predictions_and_actuals)
    if revenue > pre_revenue:
        pre_revenue = revenue
        model2_best_threshold = np.round(threshold,1)
        model2_predictions = predictions
        final_hp_predictions_and_actuals = hp_predictions_and_actuals

end2 = time.time()

revenues_list = [simulate(final_hp_predictions_and_actuals) for _ in range(1000)]
model2_revenue_std_error = std_revenue(revenues_list)

model2_time = np.round(end2-start2,2)
        
model2_revenue = np.round(pre_revenue,2)
model2_accuracy = np.round(len(model2_predictions[model2_predictions==actuals])/len(model2_predictions),2)

n_regressors.append(len(predictor_index))
revenues.append(model2_revenue)
thresholds.append(model2_best_threshold)
accuracys.append(model2_accuracy)
times.append(model2_time)
std_err_revenues.append(model2_revenue_std_error)

print('\nModel 2 parameters standard error: \n{}'.format(Model2_SE_B))
print('\nModel 2 parameters standard error < 1: \n{}'.format(Model2_SE_B[Model2_SE_B<1]))
print('\nThe Number of Selected Regressors = {}'.format(len(predictor_index)))
print('\nModel 2 Accuracy: {} '.format(model2_accuracy))
print('\nModel 2 Best Threshold: {} '.format(model2_best_threshold))
print('\nModel 2 Time: {}s'.format(model2_time))
print('\nModel 2 Revenue: ${} '.format(model2_revenue))
print('\nModel 2 Revenue Standard Error: {} '.format(model2_revenue_std_error))


Model 2 parameters standard error: 
advance_prep_required    3.063628e-01
alabama                  7.056518e+00
alaska                   1.008415e+01
alcoholic                1.698797e+08
almond                   3.904098e-01
                             ...     
cardamom                 5.246184e-01
carrot                   1.198212e-01
cashew                   2.518248e+07
casserole_gratin         3.334335e+09
cauliflower              6.898343e+08
Length: 96, dtype: float64

Model 2 parameters standard error < 1: 
advance_prep_required    0.306363
almond                   0.390410
anise                    0.319842
anniversary              0.266817
appetizer                0.099045
apple                    0.206641
apricot                  0.231898
artichoke                0.272358
arugula                  0.209511
asparagus                0.284693
australia                0.897377
avocado                  0.430873
back_to_school           0.233863
backyard_bbq             0.112729
b

### Method 3: High Correlation Columns

In [16]:
#Use all columns as regressors to find the high correlation 
x = data_balance.iloc[:, :-1]
y = data_balance.iloc[:,-1]

#Identify features with correlation to label greater than 0.1
high_corr_columns = [column for column in x.columns if abs(x[column].corr(y)) > 0.1]

""" Reapply our GLM function to columns with high correlation, then we acquire an array 
and zip to our original y vector, finally: then apply simulate function to obtain our 
new revenue estimate"""

start3 = time.time()
coefficients_high_corr = logistic_regression_fit(y, x[high_corr_columns])
B_high_corr = coefficients_high_corr.values.reshape(1,-1)[0]
Model3_SE_B = ((x[high_corr_columns]**2*np.exp(B_high_corr*x[high_corr_columns])/((1+np.exp(B_high_corr*x[high_corr_columns]))**2)).sum())**(-1/2)


# predict and evaluate the result
x = data.iloc[:, :-1]
y = data.iloc[:,-1]

threshold_candidate = np.arange(0.1,1,0.1)
pre_revenue = -np.inf
for threshold in threshold_candidate:
    predictions = predict(x[high_corr_columns], B_high_corr, threshold)
    actuals = np.array(y).copy()
    hp_predictions_and_actuals = list(zip(predictions, actuals))
    revenue = simulate(hp_predictions_and_actuals)
    if revenue > pre_revenue:
        pre_revenue = revenue
        model3_best_threshold = np.round(threshold,1)
        model3_predictions = predictions
        final_hp_predictions_and_actuals = hp_predictions_and_actuals
        
end3 = time.time()
model3_time = np.round(end3-start3,2)
        
revenues_list = [simulate(final_hp_predictions_and_actuals) for _ in range(1000)]
model3_revenue_std_error = std_revenue(revenues_list)

model3_revenue = np.round(pre_revenue,2)
model3_accuracy = np.round(len(model3_predictions[model3_predictions==actuals])/len(model3_predictions),2)

n_regressors.append(len(high_corr_columns))
revenues.append(model3_revenue)
thresholds.append(model3_best_threshold)
accuracys.append(model3_accuracy)
times.append(model3_time)
std_err_revenues.append(model3_revenue_std_error)

print('\nModel 3 parameters standard error: \n{}'.format(Model3_SE_B))
print('\nModel 3 parameters standard error < 1: \n{}'.format(Model3_SE_B[Model3_SE_B<1]))
print('\nThe Number of Selected Regressors = {}'.format(len(high_corr_columns)))
print('\nModel 3 Accuracy: {} '.format(model3_accuracy))
print('\nModel 3 Best Threshold: {} '.format(model3_best_threshold))
print('\nModel 3 Time: {}s'.format(model3_time))
print('\nModel 3 Revenue: ${} '.format(model3_revenue))
print('\nModel 3 Revenue Standard Error: {} '.format(model3_revenue_std_error))


Model 3 parameters standard error: 
alcoholic            2.040975e+16
almond               1.785671e-01
apple                1.364408e-01
bake                 3.986198e-02
bass                 2.081766e-01
bread                7.620423e+05
breakfast            1.123380e+02
brunch               9.871160e-02
cake                 5.782707e+08
cheese               1.358020e-01
chicken              5.187457e-02
chill                8.647408e-02
chocolate            9.338655e-02
clam                 1.747684e-01
cocktail_party       8.194631e-02
condiment_spread     1.214849e-01
dairy                6.749000e-02
dessert              1.175879e+06
dinner               3.522416e-02
drink                1.646854e+06
egg                  6.805343e-02
fish                 6.476502e-02
fruit                5.613960e-02
grill_barbecue       5.378000e-02
halibut              1.927060e-01
kid_friendly         5.848591e-02
kidney_friendly      5.451076e-02
kosher               3.117595e-02
lobster    

### Method 4: Chi Square Test Selection

In [17]:
#Select our X explanatory variables and response variable
x = data_balance.iloc[:, :-1]
y = data_balance.iloc[:,-1]

#Use SelectKBest function to fit
start4 = time.time()
bestfeatures = SelectKBest(score_func=chi2)

# Convert to categorical data by converting data to integers
x = x.astype(int)

fit = bestfeatures.fit(x,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['element','Score']  #naming the dataframe columns
featureScores = featureScores.sort_values(by='Score', ascending=False) # set the order from high to low

#Select the most important 50 columns based on calculated Chi-Square test
x = data.iloc[:, :-1]
y = data.iloc[:,-1]

selected_columns = featureScores[:50].element.values
coefficients_selected = logistic_regression_fit(y, x[selected_columns])
B_selected = coefficients_selected.values.reshape(1,-1)[0]
Model4_SE_B = ((x[selected_columns]**2*np.exp(B_selected*x[selected_columns])/((1+np.exp(B_selected*x[selected_columns]))**2)).sum())**(-1/2)


threshold_candidate = np.arange(0.1,1,0.1)
pre_revenue = -np.inf
for threshold in threshold_candidate:
    predictions = predict(x[selected_columns], B_selected, threshold)
    actuals = np.array(y).copy()
    hp_predictions_and_actuals = list(zip(predictions, actuals))
    revenue = simulate(hp_predictions_and_actuals)
    if revenue > pre_revenue:
        pre_revenue = revenue
        model4_best_threshold = np.round(threshold,1)
        model4_predictions = predictions
        final_hp_predictions_and_actuals = hp_predictions_and_actuals
        
end4 = time.time()
model4_time = np.round(end4-start4,2)
        
revenues_list = [simulate(final_hp_predictions_and_actuals) for _ in range(1000)]
model4_revenue_std_error = std_revenue(revenues_list)    

model4_revenue = np.round(pre_revenue,2)
model4_accuracy = np.round(len(model4_predictions[model4_predictions==actuals])/len(model4_predictions),2)

n_regressors.append(len(selected_columns))
revenues.append(model4_revenue)
thresholds.append(model4_best_threshold)
accuracys.append(model4_accuracy)
times.append(model4_time)
std_err_revenues.append(model4_revenue_std_error)

print('\nModel 4 parameters standard error: \n{}'.format(Model4_SE_B))
print('\nModel 4 parameters standard error < 1: \n{}'.format(Model4_SE_B[Model4_SE_B<1]))
print('\nThe Number of Selected Regressors = {}'.format(len(selected_columns)))
print('\nModel 4 Accuracy: {} '.format(model4_accuracy))
print('\nModel 4 Best Threshold: {} '.format(model4_best_threshold))
print('\nModel 4 Time: {}s'.format(model4_time))
print('\nModel 4 Revenue: ${} '.format(model4_revenue))
print('\nModel 4 Revenue Standard Error: {} '.format(model4_revenue_std_error))


Model 4 parameters standard error: 
vegetarian               0.143364
dessert                 35.207110
bake                     0.050376
kosher                   0.034923
side                     0.080219
turkey                   0.198095
pescatarian              0.038197
kidney_friendly          0.056313
milk_cream               0.069240
vegan                 4769.719509
fish                     0.095651
egg                      0.084514
soy_free                 0.030424
fruit                    0.061755
dairy                    0.076782
cheese                   0.122211
peanut_free              0.032949
kid_friendly             0.077585
salad                    0.080664
vegetable                0.065755
chill                    0.093684
winter                   0.052931
drink                 1103.743689
potato                   0.105778
condiment_spread         0.209354
nut                      0.148487
no_cook                  0.093142
sauce                    0.133659
roast      

### Model 5: Bootstrap & Logit Regression

In [18]:
""" This model we will bootstrap to create an equal number of high protein and low protein labels. First 
we pick out high protein rows"""

num_hight = data[data['label']==1]
num_low = data[data['label']==0]
high_random = []

start5 = time.time()
for i in range(len(num_low)-len(num_hight)):
    randon_line = num_hight.iloc[np.random.randint(len(num_hight)),:]
    high_random.append(randon_line)
bootstrap_data = data.append(high_random)

X = bootstrap_data.iloc[:,:-1]
y = bootstrap_data.iloc[:,-1]
X_p = data.iloc[:,:-1]
y_orig = data.iloc[:,-1]
clf = LogisticRegression(random_state=0).fit(X, y)

B = clf.coef_
A = clf.intercept_
B = B.flatten()
model5_prediction = (A+X_p@B)

threshold_candidate = np.arange(0.1,1,0.1)
pre_revenue = -np.inf
for threshold in threshold_candidate:
    predictions = 1 / (1 + np.exp(-model5_prediction))
    model5_prediction_sigmoid = np.where(predictions > threshold, 1, 0)
    predictions = model5_prediction_sigmoid
    actuals = np.array(y_orig).copy()
    hp_predictions_and_actuals = list(zip(predictions, actuals))
    revenue = simulate(hp_predictions_and_actuals)
    if revenue > pre_revenue:
        pre_revenue = revenue
        model5_best_threshold = np.round(threshold,1)
        model5_predictions = predictions
        final_hp_predictions_and_actuals = hp_predictions_and_actuals
        
hp_predictions_and_actuals = list(zip(model5_predictions, y_orig))       
end5 = time.time()
        
#time
model5_time = np.round(end5-start5,2)

revenues_list = [simulate(final_hp_predictions_and_actuals) for _ in range(1000)]
model5_revenue_std_error = std_revenue(revenues_list) 

#revenue
model5_revenue = simulate(hp_predictions_and_actuals)
#number of regressors
model5_N_Regressors = len(X.columns)
#accuracy
model5_predictions = [int(i) for i in model5_predictions]
model5_predictions = np.array(model5_predictions)
y_actual = np.array(y_orig)
model5_accracy = np.round((len(model5_predictions[model5_predictions == y_actual])/len(model5_predictions)),2)
#standard error
Model5_SE_B = ((X**2*np.exp(B*X)/((1+np.exp(B*X))**2)).sum())**(-1/2)

n_regressors.append(model5_N_Regressors)
revenues.append(model5_revenue)
thresholds.append(model5_best_threshold)
accuracys.append(model5_accracy)
times.append(model5_time)
std_err_revenues.append(model5_revenue_std_error)


print('\nModel 5 parameters standard error: \n{}'.format(Model5_SE_B))
print('\nModel 5 parameters standard error < 1: \n{}'.format(Model5_SE_B[Model5_SE_B<1]))
print('\nThe Number of Selected Regressors = {}'.format(model5_N_Regressors))
print('\nModel 5 Accuracy: {} '.format(model5_accracy))
print('\nModel 5 Best Threshold: {} '.format(model5_best_threshold))
print('\nModel 5 Time: {}s'.format(model5_time))
print('\nModel 5 Revenue: ${} '.format(model5_revenue))
print('\nModel 5 Revenue Standard Error: {} '.format(model5_revenue_std_error))


Model 5 parameters standard error: 
advance_prep_required    0.162691
alabama                  2.000004
alaska                   2.005975
alcoholic                0.158021
almond                   0.116390
                           ...   
cookbooks                1.414215
leftovers                1.561801
snack                    0.541421
snack_week               0.633959
turkey                   0.153052
Length: 669, dtype: float64

Model 5 parameters standard error < 1: 
advance_prep_required    0.162691
alcoholic                0.158021
almond                   0.116390
amaretto                 0.534743
anchovy                  0.534566
                           ...   
yogurt                   0.102197
zucchini                 0.138798
snack                    0.541421
snack_week               0.633959
turkey                   0.153052
Length: 516, dtype: float64

The Number of Selected Regressors = 669

Model 5 Accuracy: 0.75 

Model 5 Best Threshold: 0.1 

Model 5 Time: 9.77s



# Final Revenue Outcome and Model Detail

In [19]:
print('Model 1 has Revenue generation = ${}'.format(model1_revenue))
print('Model 2 has Revenue generation = ${}'.format(model2_revenue))
print('Model 3 has Revenue generation = ${}'.format(model3_revenue))
print('Model 4 has Revenue generation = ${}'.format(model4_revenue))
print('Model 5 has Revenue generation = ${}'.format(model5_revenue))

Model 1 has Revenue generation = $84259.25
Model 2 has Revenue generation = $84129.0
Model 3 has Revenue generation = $91721.25
Model 4 has Revenue generation = $88030.5
Model 5 has Revenue generation = $91066.25


In [20]:
columns = ['Model', 'N_Regressors', 'Revenue', 'Standard Error Revenue', 'Best Threshold', 'Accuracy', 'Time']
models = ['Random Selection','P-value Significant Regressors','High Correlation Columns',
          'Chi-Square Selection','Bootstrap & Logit Regression']
myTable = PrettyTable()

# Add Columns
myTable.add_column(columns[0], models)
myTable.add_column(columns[1], n_regressors)
myTable.add_column(columns[2], revenues)
myTable.add_column(columns[3], std_err_revenues)
myTable.add_column(columns[4], thresholds)
myTable.add_column(columns[5], accuracys)
myTable.add_column(columns[6], times)

print(myTable)

+--------------------------------+--------------+----------+------------------------+----------------+----------+--------+
|             Model              | N_Regressors | Revenue  | Standard Error Revenue | Best Threshold | Accuracy |  Time  |
+--------------------------------+--------------+----------+------------------------+----------------+----------+--------+
|        Random Selection        |     100      | 84259.25 |          0.91          |      0.4       |   0.55   |  9.48  |
| P-value Significant Regressors |      96      | 84129.0  |          0.94          |      0.1       |   0.6    | 165.11 |
|    High Correlation Columns    |      59      | 91721.25 |          0.95          |      0.2       |   0.71   | 42.63  |
|      Chi-Square Selection      |      50      | 88030.5  |          0.89          |      0.1       |   0.8    | 25.34  |
|  Bootstrap & Logit Regression  |     669      | 91066.25 |          0.96          |      0.1       |   0.75   |  9.77  |
+---------------

In [21]:
predictions_all_models = [model1_predictions,model2_predictions,model3_predictions,model4_predictions]
print('Model 1 Predict # of High protein is {}, Low protein is {}'.format(len(model1_predictions[model1_predictions==1]), len(model1_predictions[model1_predictions==0])))
print('Model 2 Predict # of High protein is {}, Low protein is {}'.format(len(model2_predictions[model2_predictions==1]), len(model2_predictions[model2_predictions==0])))
print('Model 3 Predict # of High protein is {}, Low protein is {}'.format(len(model3_predictions[model3_predictions==1]), len(model3_predictions[model3_predictions==0])))
print('Model 4 Predict # of High protein is {}, Low protein is {}'.format(len(model4_predictions[model4_predictions==1]), len(model4_predictions[model4_predictions==0])))
print(f'Model 5 Predict # of High protein is {len(model5_predictions[model5_predictions == 1])}, Low protein is {len(model5_predictions[model5_predictions == 0])}')
print('Original DataFrame: High protein = {}, Low protein = {}'.format(len(data[data.label==1]), len(data[data.label==0])))

Model 1 Predict # of High protein is 5068, Low protein is 5525
Model 2 Predict # of High protein is 4397, Low protein is 6196
Model 3 Predict # of High protein is 3557, Low protein is 7036
Model 4 Predict # of High protein is 2411, Low protein is 8182
Model 5 Predict # of High protein is 3138, Low protein is 7455
Original DataFrame: High protein = 532, Low protein = 10061


### Predict function for output samples testing

In [22]:
import random
def simulate(hp_predictions_and_actuals):
    high_protein_ad_revenue = 1
    low_protein_ad_revenue = .25
    
    ad_revenue = 0
    analysis_queue = []
    active_recipes = []
    i_data = 0
    for _ in range(365):  # for one year
        # 50-100 recipe submissions/day
        num_submissions = np.random.randint(50, 100)
        for i in range(num_submissions): # 
            if i_data < len(hp_predictions_and_actuals):
                p, a = hp_predictions_and_actuals[i_data]
                if p == 1:  # go to the front of the queue
                    analysis_queue.insert(0, a)
                    
                else: # go to the back of the queue
                    analysis_queue.append(a)
                i_data += 1
            
        # can analyze only 25 recipes/day   
        for __ in range(25):
            if len(analysis_queue)==0:
                break
            acutal = analysis_queue.pop(0)
            active_recipes.append(acutal)     
        # run 500-1000 ads/day
        num_ads_today = np.random.randint(500, 1000)
        for a in random.choices(active_recipes, k=num_ads_today):
            # a==1 if recipe is high protein
            if a==1:
                ad_revenue += high_protein_ad_revenue
            else:
                ad_revenue += low_protein_ad_revenue
    
    return ad_revenue

In [23]:
import pandas as pd
import numpy as np
np.random.seed(1)
from sklearn.linear_model import LogisticRegression
def predict(df):
    df['label']=df['protein']/df['calories']
    df['label'] = np.where(df['label']>0.1, 1, 0)
    data = df_recipe.iloc[:,3:].copy()
    num_hight = data[data['label']==1]
    num_low = data[data['label']==0]
    high_random = []
    
    for i in range(len(num_low)-len(num_hight)):
        randon_line = num_hight.iloc[np.random.randint(len(num_hight)),:]
        high_random.append(randon_line)
    bootstrap_data = data.append(high_random)

    X = bootstrap_data.iloc[:,:-1]
    y = bootstrap_data.iloc[:,-1]
    X_orig = data.iloc[:,:-1]
    clf = LogisticRegression(random_state=0,solver='lbfgs', max_iter=1000).fit(X, y)

    B = clf.coef_
    B = B.flatten()
    A = clf.intercept_
    
    model5_prediction = (A+X_orig@B)
    # using sigmoid tranform to logistic
    predictions = 1 / (1 + np.exp(-model5_prediction))
    threshold = 0.4
    predictions = np.where(predictions > threshold, 1, 0)
    
    return predictions

In [24]:
df_recipe = pd.read_csv('final_data_set.csv',index_col=0)
predict(df_recipe)

array([0, 0, 0, ..., 0, 0, 0])